****LIBRARY SEGMENT****


In [2]:
!pip install --upgrade nltk
# I got some problems about  nltk so its for upgrade

In [ ]:
######DON'T USE IT #####
import os
os._exit(00)              #it is for resetting run time


In [1]:
import pandas as pd
import os
from google.colab import drive
import shutil
from multiprocessing import Pool
import nltk
import shutil
shutil.rmtree('root/nltk_data',ignore_errors=True)
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [3]:
# for mounting Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


****DATA PREPROCESSING SEGMENT****

In [4]:

data_dir = '/content/drive/MyDrive/NLPFINALPROJECT/data-makaleler-yazarlar'

def process_file(args):
    author, text_path = args
    try:
        with open(text_path, 'r', encoding='latin-1') as f:             # i encode with latin-1 Because i get some problems when i try with utf-8
            content = f.read()                                          # i will  convert latin-1 to utf-8 in the next step
        return {'Author': author, 'Text': content}       # now we got  2 collumns Author and texts
    except:
        return None

tasks = []
for root, dirs, files in os.walk(data_dir):
    author = os.path.basename(root)
    for file in files:
        if file.endswith('.txt'):                    #we use walk for every txt that belongs a folder(author)
            text_path = os.path.join(root, file)
            tasks.append((author, text_path))

with Pool() as pool:
    data = pool.map(process_file, tasks)                #walk in files is taking so much time so i  use multiprocessing here because drive is really slow

data = [d for d in data if d is not None]  # cleaning files that has errors
df = pd.DataFrame(data)

df.head(300)   #for checking


,Author,Text
0,ATAOL BEHRAMOĞLU,Alçaklýðýn Dereceleri\n3 Mart 2012 - Cumhuriye...
1,ATAOL BEHRAMOĞLU,Yazarlarý Sokaða mý Atýyorsunuz?\n25 Þubat 201...
2,ATAOL BEHRAMOĞLU,Cumhuriyet Aydýný Hazýrlýksýz Yakalandý\n11 Þu...
3,ATAOL BEHRAMOĞLU,Bir Akýl Hastanesinde Gibi\n28 Ocak 2012 - Cum...
4,ATAOL BEHRAMOĞLU,Dinlerin Dönüþü\n21 Ocak 2012 - Cumhuriyet\nBa...
...,...,...
295,HASAN PULUR,"Laf olsun torba dolsun, diye söylenen boþ lafl..."
296,HASAN PULUR,Güzel bir deyim var: Fotoðrafa iyi bak! diyo...
297,HASAN PULUR,Acaba ceza yasasýnda þikenin ifþasý diye bir...
298,HASAN PULUR,"Modada bir yer vardý, þimdi kafe dediklerin..."


In [5]:
def convert_latin1_to_utf8(text):
    conversion_map = {
        'ý': 'ı', 'þ': 'ş', 'ð': 'ğ', 'Ý': 'İ', 'Þ': 'Ş', 'Ð': 'Ğ',
        'ç': 'ç', 'Ç': 'Ç', 'ö': 'ö', 'Ö': 'Ö', 'ü': 'ü', 'Ü': 'Ü',
        'ı': 'ı', 'İ': 'İ'
    }                                                                     # this function is for converting latin-1 to utf-8

    return ''.join(conversion_map.get(char, char) for char in text)

df['Text'] = df['Text'].apply(convert_latin1_to_utf8)     # we use the function for text  collumn


In [6]:

#this function tokenized just text
def tokenize_text(text):
    return word_tokenize(text)

#we applying to  our data frame
df['Tokens'] = df['Text'].apply(tokenize_text)

df.head()  #for checking

,Author,Text,Tokens
0,ATAOL BEHRAMOĞLU,Alçaklığın Dereceleri\n3 Mart 2012 - Cumhuriye...,"[Alçaklığın, Dereceleri, 3, Mart, 2012, -, Cum..."
1,ATAOL BEHRAMOĞLU,Yazarları Sokağa mı Atıyorsunuz?\n25 Şubat 201...,"[Yazarları, Sokağa, mı, Atıyorsunuz, ?, 25, Şu..."
2,ATAOL BEHRAMOĞLU,Cumhuriyet Aydını Hazırlıksız Yakalandı\n11 Şu...,"[Cumhuriyet, Aydını, Hazırlıksız, Yakalandı, 1..."
3,ATAOL BEHRAMOĞLU,Bir Akıl Hastanesinde Gibi\n28 Ocak 2012 - Cum...,"[Bir, Akıl, Hastanesinde, Gibi, 28, Ocak, 2012..."
4,ATAOL BEHRAMOĞLU,Dinlerin Dönüşü\n21 Ocak 2012 - Cumhuriyet\nBa...,"[Dinlerin, Dönüşü, 21, Ocak, 2012, -, Cumhuriy..."


In [7]:
df['Tokens'] = df['Tokens'].apply(lambda tokens: [t.lower() for t in tokens])        # i start to clean tokens here because i will check stopwords here i make lowercase operation
df['Tokens'] = df['Tokens'].apply(lambda tokens: [t for t in tokens if not t.isdigit()])   #cleaning numbers
import string
df['Tokens'] = df['Tokens'].apply(lambda tokens: [t for t in tokens if t not in string.punctuation])     #cleaning punctuations
#  https://github.com/sgsinclair/trombone/blob/master/src/main/resources/org/voyanttools/trombone/keywords/stop.tr.turkish-lucene.txt
#I found the turkish stopword list from this github link
turkish_stopwords = [
    "acaba", "altmış", "altı", "ama", "ancak", "arada", "aslında", "ayrıca", "bana",
    "bazı", "belki", "ben", "benden", "beni", "benim", "beri", "beş", "bile", "bin",
    "bir", "birçok", "biri", "birkaç", "birkez", "birşey", "birşeyi", "biz", "bize",
    "bizden", "bizi", "bizim", "böyle", "böylece", "bu", "buna", "bunda", "bundan",
    "bunlar", "bunları", "bunların", "bunu", "bunun", "burada", "çok", "çünkü", "da",
    "daha", "dahi", "de", "defa", "değil", "diğer", "diye", "doksan", "dokuz", "dolayı",
    "dolayısıyla", "dört", "edecek", "eden", "ederek", "edilecek", "ediliyor", "edilmesi",
    "ediyor", "eğer", "elli", "en", "etmesi", "etti", "ettiği", "ettiğini", "gibi", "göre",
    "halen", "hangi", "hatta", "hem", "henüz", "hep", "hepsi", "her", "herhangi",
    "herkesin", "hiç", "hiçbir", "için", "iki", "ile", "ilgili", "ise", "işte", "itibaren",
    "itibariyle", "kadar", "karşın", "katrilyon", "kendi", "kendilerine", "kendini",
    "kendisi", "kendisine", "kendisini", "kez", "ki", "kim", "kimden", "kime", "kimi",
    "kimse", "kırk", "milyar", "milyon", "mu", "mü", "mı", "nasıl", "ne", "neden",
    "nedenle", "nerde", "nerede", "nereye", "niye", "niçin", "o", "olan", "olarak",
    "oldu", "olduğu", "olduğunu", "olduklarını", "olmadı", "olmadığı", "olmak",
    "olması", "olmayan", "olmaz", "olsa", "olsun", "olup", "olur", "olursa", "oluyor",
    "on", "ona", "ondan", "onlar", "onlardan", "onları", "onların", "onu", "onun",
    "otuz", "oysa", "öyle", "pek", "rağmen", "sadece", "sanki", "sekiz", "seksen",
    "sen", "senden", "seni", "senin", "siz", "sizden", "sizi", "sizin", "şey",
    "şeyden", "şeyi", "şeyler", "şöyle", "şu", "şuna", "şunda", "şundan", "şunları",
    "şunu", "tarafından", "trilyon", "tüm", "üç", "üzere", "var", "vardı", "ve",
    "veya", "ya", "yani", "yapacak", "yapılan", "yapılması", "yapıyor", "yapmak",
    "yaptı", "yaptığı", "yaptığını", "yaptıkları", "yedi", "yerine", "yetmiş", "yine",
    "yirmi", "yoksa", "yüz", "zaten","sonra", "önce", "zaman", "yıl", "bugün", "dün", "artık", "ilk", "şimdi"
]
# here we are using stopwords to filtering out our text.
def remove_stopwords(tokens):
    return [t for t in tokens if t not in turkish_stopwords]

df['Tokens'] = df['Tokens'].apply(remove_stopwords)    #function call  for tokens
df.head()

,Author,Text,Tokens
0,ATAOL BEHRAMOĞLU,Alçaklığın Dereceleri\n3 Mart 2012 - Cumhuriye...,"[alçaklığın, dereceleri, mart, cumhuriyet, alç..."
1,ATAOL BEHRAMOĞLU,Yazarları Sokağa mı Atıyorsunuz?\n25 Şubat 201...,"[yazarları, sokağa, atıyorsunuz, şubat, cumhur..."
2,ATAOL BEHRAMOĞLU,Cumhuriyet Aydını Hazırlıksız Yakalandı\n11 Şu...,"[cumhuriyet, aydını, hazırlıksız, yakalandı, ş..."
3,ATAOL BEHRAMOĞLU,Bir Akıl Hastanesinde Gibi\n28 Ocak 2012 - Cum...,"[akıl, hastanesinde, ocak, cumhuriyet, i̇çleri..."
4,ATAOL BEHRAMOĞLU,Dinlerin Dönüşü\n21 Ocak 2012 - Cumhuriyet\nBa...,"[dinlerin, dönüşü, ocak, cumhuriyet, başlığı, ..."


In [8]:
from collections import Counter

#combining all tokens
all_tokens = [token for tokens in df['Tokens'] for token in tokens]
                                                                          # here is for examine which stop words or useless words may cause problem in training
# most often word  in the text
word_counts = Counter(all_tokens)
print(word_counts.most_common(50))


[('...', 8409), ('büyük', 1426), ("''", 1388), ('\x94', 1372), ('mi', 1363), ('..', 1346), ('son', 1295), ('yok', 1281), ('``', 1222), ('yeni', 1138), ('iyi', 1127), ('başka', 1102), ('aynı', 1085), ('içinde', 1051), ('gün', 1022), ('bütün', 1001), ('önemli', 884), ('karşı', 850), ('tek', 808), ('doğru', 677), ('biraz', 676), ('söz', 673), ('insan', 661), ('arasında', 645), ('tam', 624), ('ortaya', 616), ('türkiye', 602), ('yer', 600), ('hemen', 567), ('kadın', 566), ('birlikte', 562), ('devlet', 556), ('türk', 543), ('uzun', 541), ('eski', 533), ('fazla', 527), ('yüzde', 520), ('geçen', 506), ('üzerine', 504), ('özel', 501), ('güzel', 488), ('dünya', 488), ('gelen', 482), ('devam', 464), ('başbakan', 448), ('genel', 440), ('vardır', 435), ('kabul', 425), ('fakat', 420), ('farklı', 419)]


In [9]:
import re

def clean_tokens(tokens):
    return [t for t in tokens if not re.match(r"^[\W_]+$", t)]  #we got some problems about special characthers so i removed

df['Tokens'] = df['Tokens'].apply(clean_tokens)


In [ ]:
df.head()

,Author,Text,Tokens
0,ATAOL BEHRAMOĞLU,Alçaklığın Dereceleri\n3 Mart 2012 - Cumhuriye...,"[alçaklığın, dereceleri, mart, cumhuriyet, alç..."
1,ATAOL BEHRAMOĞLU,Yazarları Sokağa mı Atıyorsunuz?\n25 Şubat 201...,"[yazarları, sokağa, atıyorsunuz, şubat, cumhur..."
2,ATAOL BEHRAMOĞLU,Cumhuriyet Aydını Hazırlıksız Yakalandı\n11 Şu...,"[cumhuriyet, aydını, hazırlıksız, yakalandı, ş..."
3,ATAOL BEHRAMOĞLU,Bir Akıl Hastanesinde Gibi\n28 Ocak 2012 - Cum...,"[akıl, hastanesinde, ocak, cumhuriyet, i̇çleri..."
4,ATAOL BEHRAMOĞLU,Dinlerin Dönüşü\n21 Ocak 2012 - Cumhuriyet\nBa...,"[dinlerin, dönüşü, ocak, cumhuriyet, başlığı, ..."


This step , i wont use any ready model because we try to  extract context in all texts. So i want to train my own model .

In [29]:
from gensim.models import Word2Vec

# training word2vec model
word2vec_model = Word2Vec(
    sentences=df['Tokens'],  # tokenized clean texts
    vector_size=100,
    window=5,
    min_count=2,             #minimum word count that model conider
    workers=4,               # threads count for multiprocessing
    sg=1                     # skip gram model for learning context well
)

# saving model
word2vec_model.save("word2vec_yazar.model")


In [30]:
import numpy as np

def document_vector(tokens, model):
    # we get vectors  with model
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) == 0:
        return np.zeros(model.vector_size)                                        #now we  are recording vectors to a new collumn
    return np.mean(vectors, axis=0)

#here we are creating new collumn and applying the function
df['Doc_Vector'] = df['Tokens'].apply(lambda tokens: document_vector(tokens, word2vec_model))

df.head()

,Tokens,Author,Author_Label,Doc_Vector
0,"[alçaklığın, dereceleri, mart, cumhuriyet, alç...",ATAOL BEHRAMOĞLU,2,"[-0.027570533, 0.24713452, 0.12275129, -0.0495..."
1,"[yazarları, sokağa, atıyorsunuz, şubat, cumhur...",ATAOL BEHRAMOĞLU,2,"[-0.01863111, 0.22950655, 0.083701394, -0.0452..."
2,"[cumhuriyet, aydını, hazırlıksız, yakalandı, ş...",ATAOL BEHRAMOĞLU,2,"[-0.011899525, 0.23730679, 0.08749661, -0.0502..."
3,"[akıl, hastanesinde, ocak, cumhuriyet, i̇çleri...",ATAOL BEHRAMOĞLU,2,"[-0.024413472, 0.2387065, 0.12177619, -0.04253..."
4,"[dinlerin, dönüşü, ocak, cumhuriyet, başlığı, ...",ATAOL BEHRAMOĞLU,2,"[-0.03177987, 0.26525238, 0.10633631, -0.03685..."


In [12]:
import numpy as np

#  Storing each document separately with its vector representation
X_1500 = np.vstack(df['Doc_Vector'].values)  # 1500 document vectors
y_1500 = df['Author'].values  # 1500 labels (distributed among 30 authors)

#  Print dataset shape for verification
print("#1500 label based data shape:")
print("X_1500 shape:", X_1500.shape)  # Expected shape: (1500, 100) → 1500 samples, each with a 100-dim vector
print("y_1500 shape:", y_1500.shape)  # Expected shape: (1500,) → 1500 labels corresponding to authors


#1500 label based data shape:
X_1500 shape: (1500, 100)
y_1500 shape: (1500,)


In [13]:
# Get the average document vector for each author
author_vectors = df.groupby('Author')['Doc_Vector'].apply(lambda x: np.mean(np.vstack(x), axis=0)).reset_index()

# Convert to arrays for model training
X_30 = np.vstack(author_vectors['Doc_Vector'].values)  # Feature matrix (30 authors)
y_30 = author_vectors['Author'].values  # Labels (author names)

# Print shapes for verification
print("\n#30 label based data shape:")
print("X_30 shape:", X_30.shape)  # (30, 100)
print("y_30 shape:", y_30.shape)  # (30,)



#30 label based data shape:
X_30 shape: (30, 100)
y_30 shape: (30,)


In [14]:
from sklearn.model_selection import StratifiedKFold

# Create 5-Fold splits with stratification
skf_1500 = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Generate fold indices for 1500 samples
folds_1500 = []
for train_idx, test_idx in skf_1500.split(X_1500, y_1500):
    folds_1500.append((train_idx, test_idx))

# Print fold sizes for verification
for i, (train_idx, test_idx) in enumerate(folds_1500):
    print(f"\n#1500 label based - Fold {i+1}:")
    print(f"Train set: {len(train_idx)}, Test set: {len(test_idx)}")



#1500 label based - Fold 1:
Train set: 1200, Test set: 300

#1500 label based - Fold 2:
Train set: 1200, Test set: 300

#1500 label based - Fold 3:
Train set: 1200, Test set: 300

#1500 label based - Fold 4:
Train set: 1200, Test set: 300

#1500 label based - Fold 5:
Train set: 1200, Test set: 300


In [15]:
from sklearn.model_selection import train_test_split
import numpy as np

# Randomly select 200 samples as the test set from 1500 samples
X_train_1500, X_test_1500, y_train_1500, y_test_1500 = train_test_split(
    X_1500, y_1500, test_size=200/1500, stratify=y_1500, random_state=42  # Stratified split to maintain class balance
)

# Print the new dataset shapes
print("New Training Set (from 1500 samples):", X_train_1500.shape)
print("New Test Set (from 1500 samples):", X_test_1500.shape)

# Compute the average document vector for each author
author_vectors = df.groupby('Author')['Doc_Vector'].apply(lambda x: np.mean(np.vstack(x), axis=0)).reset_index()

# Convert to feature matrix (30 authors, each represented by one vector)
X_train_30 = np.vstack(author_vectors['Doc_Vector'].values)
y_train_30 = author_vectors['Author'].values

# Print dataset shapes for verification
print("30 Label Based Training Set:", X_train_30.shape)
print("30 Label Based Author Labels:", y_train_30.shape)


New Training Set (from 1500 samples): (1300, 100)
New Test Set (from 1500 samples): (200, 100)
30 Label Based Training Set: (30, 100)
30 Label Based Author Labels: (30,)


HERE   I GET BEST PERFORMANCE SO  I USE IT FOR FINAL REPORT


****Training and examine results segment****

In [16]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.decomposition import PCA

#  Lists to store accuracy and classification reports for each fold
accuracies = []
all_reports = []

#  5-Fold Cross Validation Loop
for i, (train_idx, test_idx) in enumerate(folds_1500):
    print(f"\n Fold {i+1} training start...")

    # Splitting the dataset into train and test sets for this fold
    X_train, X_test = X_1500[train_idx], X_1500[test_idx]
    y_train, y_test = y_1500[train_idx], y_1500[test_idx]

    # Reducing dimensionality with PCA (100 → 50)
    pca = PCA(n_components=50)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)

    #  Neural Network Model
    mlp_model = MLPClassifier(
        hidden_layer_sizes=(256, 128, 64),  # Multi-layer perceptron with 3 hidden layers
        activation='relu',  # ReLU activation function
        max_iter=1000,  # Longer training for better convergence
        learning_rate_init=0.001,  # Suitable learning rate for Adam optimizer
        solver='adam',  # Stable optimization method
        random_state=42  # Ensuring reproducibility
    )

    # Train the model
    mlp_model.fit(X_train_pca, y_train)

    # Predict on the test set
    y_pred = mlp_model.predict(X_test_pca)

    # Compute accuracy
    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)

    # Store classification report for this fold
    report = classification_report(y_test, y_pred, output_dict=True)
    all_reports.append(report)

    print(f"✅ Fold {i+1} completed! Accuracy: {acc:.4f}")

#  Compute Mean Accuracy
mean_acc = sum(accuracies) / len(accuracies)
print(f"\n🔥 Mean Accuracy (MLP, 1500 label based): {mean_acc:.4f}")

#  Precision, Recall, and F1-Score Table
class_labels = sorted(set(y_1500))  # List of unique author labels
precision_vals, recall_vals, f1_vals = [], [], []

# Compute average metrics across all folds
for label in class_labels:
    prec = np.mean([report[str(label)]["precision"] for report in all_reports])
    rec = np.mean([report[str(label)]["recall"] for report in all_reports])
    f1 = np.mean([report[str(label)]["f1-score"] for report in all_reports])

    precision_vals.append(prec)
    recall_vals.append(rec)
    f1_vals.append(f1)

# Compute overall mean values
precision_vals.append(np.mean(precision_vals))
recall_vals.append(np.mean(recall_vals))
f1_vals.append(np.mean(f1_vals))

#  Create Final Performance Table
performance_df = pd.DataFrame({
    "Class": class_labels + ["Average"],
    "Precision": precision_vals,
    "Recall": recall_vals,
    "F-Score": f1_vals
})

# Save results as CSV
performance_df.to_csv("MLP_Performance_Results.csv", index=False)

# Save results as LaTeX format
performance_df.to_latex("MLP_Performance_Results.tex", index=False)

# Display first 1000 rows (for verification)
performance_df.head(1000)



 Fold 1 training start...


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


✅ Fold 1 completed! Accuracy: 0.5400

 Fold 2 training start...


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


✅ Fold 2 completed! Accuracy: 0.4933

 Fold 3 training start...


...

✅ Fold 3 completed! Accuracy: 0.5000

 Fold 4 training start...


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


✅ Fold 4 completed! Accuracy: 0.5067

 Fold 5 training start...
✅ Fold 5 completed! Accuracy: 0.5533

🔥 Mean Accuracy (MLP, 1500 label based): 0.5187


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


,Class,Precision,Recall,F-Score
0,AHMET ÇAKAR,0.751111,0.780000,0.762010
1,ALİ SİRMEN,0.425079,0.320000,0.361858
2,ATAOL BEHRAMOĞLU,0.472308,0.500000,0.475348
3,ATİLLA DORSAY,0.620606,0.700000,0.650754
4,AYKAN SEVER,0.489394,0.560000,0.521645
5,AZİZ ÜSTEL,0.510256,0.540000,0.522363
6,CAN ATAKLI,0.702914,0.560000,0.582262
7,DENİZ GÖKÇE,0.930303,0.900000,0.902244
8,EMRE KONGAR,0.410684,0.400000,0.388784
9,GÖZDE BEDELOĞLU,0.470909,0.420000,0.441905


In [ ]:
df.head()

,Author,Text,Tokens,Doc_Vector
0,ATAOL BEHRAMOĞLU,Alçaklığın Dereceleri\n3 Mart 2012 - Cumhuriye...,"[alçaklığın, dereceleri, mart, cumhuriyet, alç...","[-0.033462174, 0.24169296, 0.119772375, -0.052..."
1,ATAOL BEHRAMOĞLU,Yazarları Sokağa mı Atıyorsunuz?\n25 Şubat 201...,"[yazarları, sokağa, atıyorsunuz, şubat, cumhur...","[-0.018586414, 0.2262276, 0.08191353, -0.04283..."
2,ATAOL BEHRAMOĞLU,Cumhuriyet Aydını Hazırlıksız Yakalandı\n11 Şu...,"[cumhuriyet, aydını, hazırlıksız, yakalandı, ş...","[-0.019256147, 0.23043205, 0.08620853, -0.0503..."
3,ATAOL BEHRAMOĞLU,Bir Akıl Hastanesinde Gibi\n28 Ocak 2012 - Cum...,"[akıl, hastanesinde, ocak, cumhuriyet, i̇çleri...","[-0.027514638, 0.23214857, 0.123391904, -0.042..."
4,ATAOL BEHRAMOĞLU,Dinlerin Dönüşü\n21 Ocak 2012 - Cumhuriyet\nBa...,"[dinlerin, dönüşü, ocak, cumhuriyet, başlığı, ...","[-0.037060615, 0.2572421, 0.10403246, -0.03696..."


****TRYOUTS ON LLMS****

In [17]:
!pip install transformers datasets torch accelerate
# requirement for llms
#  i couldnt download gcsfs  but  i learned it is for cloud not about this project

In [18]:
import torch
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# ✅ Check if GPU is available, otherwise use CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")  # Display the device being used

# ✅ Load BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)  # Tokenizer for text processing
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=30)  # Model with 30 output classes (authors)
model.to(device)  # Move model to GPU if available


Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [19]:
# Select only necessary columns (Tokens and Author)
df = df[["Tokens", "Author"]]

# ✅ Convert author names to numeric labels (0-29)
label_encoder = LabelEncoder()
df["Author_Label"] = label_encoder.fit_transform(df["Author"])

# ✅ Split dataset into training (80%) and test (20%) sets, ensuring class balance
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["Author_Label"], random_state=42)

# ✅ Convert data into Hugging Face dataset format for compatibility with transformers
train_dataset = Dataset.from_pandas(train_df[["Tokens", "Author_Label"]])
test_dataset = Dataset.from_pandas(test_df[["Tokens", "Author_Label"]])


<ipython-input-19-23a2f03d14ee>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Author_Label"] = label_encoder.fit_transform(df["Author"])


In [20]:
# Convert dataset to a format compatible with BERT
def format_dataset(example):
    text = " ".join(example["Tokens"])  # Convert token list into a single string
    encoded = tokenizer(text, truncation=True, padding="max_length", max_length=512, return_tensors="pt")
    return {
        "input_ids": encoded["input_ids"].squeeze(0).tolist(),  # Convert tensor to list
        "attention_mask": encoded["attention_mask"].squeeze(0).tolist(),
        "labels": int(example["Author_Label"])  # Convert label to integer
    }

# ✅ Apply transformation to datasets
train_dataset = train_dataset.map(format_dataset)
test_dataset = test_dataset.map(format_dataset)

# Remove unnecessary columns
columns_to_remove = ["Tokens", "Author_Label", "__index_level_0__"]
for col in columns_to_remove:
    if col in train_dataset.column_names:
        train_dataset = train_dataset.remove_columns(col)
    if col in test_dataset.column_names:
        test_dataset = test_dataset.remove_columns(col)

# Display dataset structure for verification
print("📌 Train Dataset Columns:", train_dataset.column_names)
print("📌 Test Dataset Columns:", test_dataset.column_names)


Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

📌 Train Dataset Columns: ['input_ids', 'attention_mask', 'labels']
📌 Test Dataset Columns: ['input_ids', 'attention_mask', 'labels']


In [21]:
# Apply padding dynamically to batches
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
################# i try it so many times and i decieded in  this  trainin epoch number
training_args = TrainingArguments(
    output_dir="./results",  # Directory to save model checkpoints
    evaluation_strategy="epoch",  # Evaluate model at the end of each epoch
    learning_rate=2e-5,  # Lower learning rate for stable training
    per_device_train_batch_size=4,  # Batch size per device during training
    per_device_eval_batch_size=4,  # Batch size per device during evaluation
    num_train_epochs=10,  # Number of training epochs
    weight_decay=0.01,  # Regularization to prevent overfitting
    save_strategy="epoch",  # Save model checkpoint after each epoch
    logging_steps=50,  # Log training progress every 50 steps
    fp16=torch.cuda.is_available(),  # Use mixed precision if running on GPU
    push_to_hub=False,  # Disable automatic model upload
    remove_unused_columns=False,  # Keep all dataset columns
    group_by_length=False  # Avoid sorting sequences by length
)

#nitialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator
)

# Train the model
trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,3.445400,3.407187
2,3.159000,3.098389
3,2.874600,2.809246
4,2.765300,2.729700
5,2.625100,2.694551
6,2.419600,2.510897
7,2.264400,2.380915
8,2.254500,2.296607
9,2.148300,2.244529
10,2.149200,2.229401


TrainOutput(global_step=3000, training_loss=2.6617075525919596, metrics={'train_runtime': 322.6826, 'train_samples_per_second': 37.188, 'train_steps_per_second': 9.297, 'total_flos': 3158126419968000.0, 'train_loss': 2.6617075525919596, 'epoch': 10.0})

In [22]:
model.save_pretrained("./bert_yazar_modeli")
tokenizer.save_pretrained("./bert_yazar_modeli")   #saving model  because it takes time a lot
print(" Model Saved!")


 Model Saved!


In [23]:

#saving model to drive too
save_directory = "/content/drive/MyDrive/bert_yazar_modeli"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print(f"model saved to  {save_directory}")


model saved to  /content/drive/MyDrive/bert_yazar_modeli


In [24]:
import numpy as np
from sklearn.metrics import accuracy_score

# we use it our model again
model.to(device)

# taking predictions
predictions = trainer.predict(test_dataset)
logits = predictions.predictions
predicted_labels = np.argmax(logits, axis=-1)

# ✅  we take the real labels
true_labels = np.array(test_dataset["labels"])

# ✅ counting accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"📊 Final Test Accuracy: {accuracy:.4f}")


📊 Final Test Accuracy: 0.3300


THIS ONE IS FOR GPT 2

In [25]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from datasets import Dataset

#  Check if GPU is available, otherwise use CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")  # Display the selected device

# Load GPT-2 model and tokenizer
model_name = "openai-community/gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Fix padding issue for GPT-2 (GPT-2 does not have a default padding token)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Ensure padding is applied to the right

#  Load GPT-2 model for sequence classification (30 author labels)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=30)
model.config.pad_token_id = tokenizer.pad_token_id  # Assign the pad token to the model
model.to(device)  # Move model to GPU if available
#  Function to tokenize dataset
def format_dataset(example):
    if "input_ids" in example and "attention_mask" in example:
        return example  # Skip processing if already tokenized

    text = " ".join(example["Tokens"]) if isinstance(example["Tokens"], list) else example["Tokens"]
    encoded = tokenizer(text, truncation=True, padding="max_length", max_length=512, return_tensors="pt")

    return {
        "input_ids": encoded["input_ids"].squeeze(0).tolist(),
        "attention_mask": encoded["attention_mask"].squeeze(0).tolist(),
        "labels": int(example["Author_Label"])
    }

# Apply tokenization to datasets if not already processed
if "input_ids" not in train_dataset.column_names:
    train_dataset = train_dataset.map(format_dataset)
if "input_ids" not in test_dataset.column_names:
    test_dataset = test_dataset.map(format_dataset)

# Remove unnecessary columns
columns_to_remove = ["Tokens", "Author_Label", "__index_level_0__"]
for col in columns_to_remove:
    if col in train_dataset.column_names:
        train_dataset = train_dataset.remove_columns(col)
    if col in test_dataset.column_names:
        test_dataset = test_dataset.remove_columns(col)

# Print dataset structure
print(" Train Dataset Columns:", train_dataset.column_names)
print(" Test Dataset Columns:", test_dataset.column_names)

# Add padding for dynamic batch sizes
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#  Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Evaluate after each epoch
    learning_rate=5e-5,  # Default learning rate
    per_device_train_batch_size=1,  # Training batch size
    per_device_eval_batch_size=1,  # Evaluation batch size
    num_train_epochs=3,  # Number of epochs
    weight_decay=0.01,  # Regularization
    save_strategy="epoch",  # Save model checkpoints at each epoch
    logging_steps=50,  # Log progress every 50 steps
    fp16=torch.cuda.is_available(),  # Use FP16 if running on GPU
    push_to_hub=False,  # Disable model upload
    remove_unused_columns=False,  # Ensure all dataset columns are retained
    group_by_length=False  # Avoid sorting sequences by length
)

#  Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator
)

# Train the model
trainer.train()


Using device: cuda


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


 Train Dataset Columns: ['input_ids', 'attention_mask', 'labels']
 Test Dataset Columns: ['input_ids', 'attention_mask', 'labels']


Epoch,Training Loss,Validation Loss
1,3.382100,3.273214
2,3.266100,3.193186
3,3.045700,3.084475


TrainOutput(global_step=3600, training_loss=3.3043230141533746, metrics={'train_runtime': 185.05, 'train_samples_per_second': 19.454, 'train_steps_per_second': 19.454, 'total_flos': 940906119168000.0, 'train_loss': 3.3043230141533746, 'epoch': 3.0})

In [26]:

#saving models to drive
save_directory = "/content/drive/MyDrive/fine_tuned_gpt2"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print(f" directory of model: {save_directory}")


 directory of model: /content/drive/MyDrive/fine_tuned_gpt2


In [ ]:
df.head()

,Tokens,Author,Author_Label
0,"[alçaklığın, dereceleri, mart, cumhuriyet, alç...",ATAOL BEHRAMOĞLU,2
1,"[yazarları, sokağa, atıyorsunuz, şubat, cumhur...",ATAOL BEHRAMOĞLU,2
2,"[cumhuriyet, aydını, hazırlıksız, yakalandı, ş...",ATAOL BEHRAMOĞLU,2
3,"[akıl, hastanesinde, ocak, cumhuriyet, i̇çleri...",ATAOL BEHRAMOĞLU,2
4,"[dinlerin, dönüşü, ocak, cumhuriyet, başlığı, ...",ATAOL BEHRAMOĞLU,2


In [27]:
import numpy as np
from sklearn.metrics import accuracy_score

# Get model predictions on the test set
predictions = trainer.predict(test_dataset)

# Convert model output probabilities into class labels
preds = np.argmax(predictions.predictions, axis=-1)  # Select the class with the highest probability

# Retrieve true labels from the dataset
labels = np.array(test_dataset["labels"])  # "labels" column contains true author labels

# Compute accuracy
accuracy = accuracy_score(labels, preds)
print(f"📊 Test Accuracy: {accuracy:.4f}")  # Display accuracy with four decimal places


📊 Test Accuracy: 0.1000


****LLM TRY IS OVER****

i need to go baCk and run some cells about vectorization operations because our df table is changed on LLM section

In [28]:
df.head()

,Tokens,Author,Author_Label
0,"[alçaklığın, dereceleri, mart, cumhuriyet, alç...",ATAOL BEHRAMOĞLU,2
1,"[yazarları, sokağa, atıyorsunuz, şubat, cumhur...",ATAOL BEHRAMOĞLU,2
2,"[cumhuriyet, aydını, hazırlıksız, yakalandı, ş...",ATAOL BEHRAMOĞLU,2
3,"[akıl, hastanesinde, ocak, cumhuriyet, i̇çleri...",ATAOL BEHRAMOĞLU,2
4,"[dinlerin, dönüşü, ocak, cumhuriyet, başlığı, ...",ATAOL BEHRAMOĞLU,2


SVM

In [34]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
#  Define hyperparameter grid for tuning
param_grid = {
    "C": [0.1, 1, 10],  # Regularization parameter (controls margin size)
    "kernel": ["linear", "rbf", "poly"],  # Different kernel types
    "gamma": ["scale", "auto"]  # Gamma parameter for RBF and poly kernels
}

#  Perform GridSearchCV to find the best SVM model
svm_grid = GridSearchCV(SVC(), param_grid, cv=5, scoring="accuracy", n_jobs=-1)  # 5-fold cross-validation
svm_grid.fit(X_train, y_train)

#  Retrieve the best model and parameters
best_svm = svm_grid.best_estimator_
best_params = svm_grid.best_params_

# Predict using the best model
y_pred_best = best_svm.predict(X_test)

# Compute accuracy
best_accuracy = accuracy_score(y_test, y_pred_best)

print(f"✅ Best SVM Model: {best_params}")
print(f"📊 Best SVM Accuracy: {best_accuracy:.4f}")


✅ Best SVM Model: {'C': 10, 'gamma': 'scale', 'kernel': 'poly'}
📊 Best SVM Accuracy: 0.4600


SVM  with 30label based

In [35]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# ✅ Define SVM model with manually chosen parameters
best_svm = SVC(kernel="linear", C=1.0)  # Linear kernel with regularization C=1.0

# ✅ Train the model using the full dataset (No cross-validation since each class has only one example)
best_svm.fit(X_30, y_30)

# ✅ Predict on the test set
y_pred_best = best_svm.predict(X_test)

# ✅ Compute accuracy
best_accuracy = accuracy_score(y_test, y_pred_best)
print(f"📊 Final SVM Accuracy: {best_accuracy:.4f}")


📊 Final SVM Accuracy: 0.4133


BAYES

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Split dataset into train (80%) and test (20%) while maintaining class balance
X_train, X_test, y_train, y_test = train_test_split(X_1500, y_1500, test_size=0.2, stratify=y_1500, random_state=42)

#  Initialize Gaussian Naive Bayes model
nb_model = GaussianNB()

#  Train the model on the training set
nb_model.fit(X_train, y_train)

#  Predict on the test set
y_pred = nb_model.predict(X_test)

# Compute accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f" Naive Bayes Model (1500 Label Based) Test Accuracy: {accuracy:.4f}")


 Naive Bayes Model (1500 Label Based) Test Accuracy: 0.3933


In [38]:
from sklearn.model_selection import cross_val_score

# Gaussian Naive Bayes with Cross Validation
cv_scores = cross_val_score(GaussianNB(), X_1500, y_1500, cv=5, scoring="accuracy")
print(f"Cross Validation Mean Accuracy (5-Fold): {np.mean(cv_scores):.4f}")


Cross Validation Mean Accuracy (5-Fold): 0.3827
